# Subsea Inpanting using FGVC ([paper](https://arxiv.org/abs/2009.01835), [code](https://github.com/vt-vl-lab/FGVC))
 - Necessary to enable GPU. Menu options −> Runtime -> Change runtime type

# 1. Setup

## 1.1. Mount your google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 1.2. Link the dataset to your google drive
- You can link from [my google drive](https://drive.google.com/file/d/1OaTLKxkgKlAXMD4PFeHu4YxVVR_nqrkL/view?usp=sharing)
  - Or you can manually download the zip from google drive (or from [kaggle datasets](https://www.kaggle.com/brunomsantiago/subsea-inpainting-dataset)) and then upload to your google drive.
  - In both cases the zip should be be at your google drive root.
- Unizp the dataset using the commented code below
- You have to unzip this only once, therefore the cell bellow is commented by default

In [ ]:
# %%bash
# cd drive/MyDrive
# mkdir subsea_inpainting
# unzip subsea_inpainting_dataset.zip -d subsea_inpainting/dataset

## 1.3. Install [viajen](https://github.com/brunomsantiago/viajen) for animation visualization

In [ ]:
!pip install viajen

## 1.4. Download deepfill code and models from [subsea inpainting repository](https://github.com/brunomsantiago/subsea_inpainting)
- You have to download this only once, therefore the cell bellow is commented by default

In [ ]:
# %%bash
# cd drive/MyDrive/subsea_inpainting
# git clone https://github.com/brunomsantiago/subsea_inpainting --branch models

## 1.5. Normal imports

In [ ]:
# Standard Library imports
from pathlib import Path
import sys
import time

# Third party imports
import cv2
from IPython.core.display import HTML
import numpy as np
from PIL import Image
from viajen import animate

## 1.6 Local import (non-installed module)

In [ ]:
base_models_folder = Path('/content/drive/MyDrive/subsea_inpainting/subsea_inpainting')
model_code_folder = base_models_folder / 'code' / 'FGVC' / 'tool'
sys.path.insert(1, model_code_folder.as_posix())
from video_completion import video_completion_seamless

## 1.7. Custom dictionary to emulate argparse results

In [ ]:
class AttributeDict(dict):
    '''
    Dict which keys can be accessed like a normal dict:
        my_dict['key']
    or like a class:
        my_dict.key
    Useful to mimic the behaviour of argparse results
    '''
    def __getattr__(self, attr):
        return self[attr]
    def __setattr__(self, attr, value):
        self[attr] = value
    def _get_kwargs(self):
        return sorted(self.__dict__.items())

# 2. Frames to be inpainted (+ masks)

## 2.1. Select clip to work with

In [ ]:
data_name = '01a'
# data_name = '01b'
# data_name = '01c'
# data_name = '02a'
# data_name = '03a'
# data_name = '03b'
# data_name = '04a'
# data_name = '05a'
# data_name = '05b'
# data_name = '05c'
# data_name = '05d'
# data_name = '05e'
# data_name = '05f'

## 2.2. Mount folders

In [ ]:
method = 'fgvc'
base_input_folder = '/content/drive/MyDrive/subsea_inpainting/dataset'
base_output_folder = '/content/drive/MyDrive/subsea_inpainting/results/static'

frames_folder = Path(base_input_folder) / 'prepared_images' / data_name / 'frames'
masks_folder = Path(base_input_folder) / 'prepared_images' / data_name / 'masks'

results_folder = Path(base_output_folder) / data_name / method

## 2.3. View clip

In [ ]:
animate.folder(frames_folder, max_height=200)

In [ ]:
animate.folder(masks_folder, max_height=200)

# 3. Inpaiting

### 3.1. Args for video completion

In [ ]:
raft_weights = base_models_folder / 'models_weights' / 'fgvc' / 'raft-things.pth'
deepfill_weights = base_models_folder / 'models_weights' / 'fgvc' / 'imagenet_deepfill.pth'
edge_completion_weights = base_models_folder / 'models_weights' / 'fgvc' / 'edge_completion.pth'

args = AttributeDict()
# Video completion
args['seamless'] = True
args['edge_guide'] = False
args['mode'] = 'object_removal'
args['path'] = frames_folder
args['path_mask'] = masks_folder
args['outroot'] = results_folder
args['consistencyThres'] = np.inf
args['alpha'] = 0.1
args['Nonlocal'] = False
# RAFT
args['model'] = raft_weights.as_posix()
args['small'] = False
args['mixed_precision'] = False
args['alternate_corr'] = False
# Deepfill
args['deepfill_model'] = deepfill_weights.as_posix()
# Edge completion
args['edge_completion_model'] = edge_completion_weights.as_posix()

## 3.2. Function to apply inpainting

In [ ]:
def inpaint_fgvc(args):
    start_time = time.time()     
    video_completion_seamless(args)
    return time.time() - start_time

## 3.3. Execute inpainting

In [ ]:
execution_time = inpaint_fgvc(args)
print(f'\n\n--- Execution time: {execution_time:.2f} seconds ---')

# 4. Show results

In [ ]:
display(HTML('<h2>Original Frames<h2>'))
display(animate.folder(frames_folder))

display(HTML(f'<br><br><h2>Inpainted Frames ({execution_time:.1f} seconds processing)<h2>'))
display(animate.folder(results_folder / 'frame_seamless_comp_final'))